 create other relevant features using data that can be obtained in the public domain

In [8]:
import pandas as pd
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm import tqdm
import time
import pickle

In [2]:
df1=pd.read_csv("/home/moraa/Documents/10_academy/Week-8/Data/driver_locations_during_request.csv")

In [4]:
df1.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN


In [3]:
df2=pd.read_csv("/home/moraa/Documents/10_academy/Week-8/Data/nb.csv")

In [5]:
df2.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


The Time Frame from Trip Data

In [6]:
import pandas as pd

# Convert date columns to datetime format
df2['Trip Start Time'] = pd.to_datetime(df2['Trip Start Time'])
df2['Trip End Time'] = pd.to_datetime(df2['Trip End Time'])

# Determine the time frame from trip data
start_date = df2['Trip Start Time'].min()
end_date = df2['Trip End Time'].max()

print(f"Data should be gathered between {start_date} and {end_date}.")


Data should be gathered between 2021-07-01 06:21:02 and 2022-06-13 10:42:15.


The Unique Locations from the Datasets

In [7]:
# Extract unique locations from df1
driver_locations = df1[['lat', 'lng']].drop_duplicates()

# Extract origin and destination locations from df2
trip_origins = df2['Trip Origin'].str.split(',', expand=True).astype(float).rename(columns={0: 'lat', 1: 'lng'})
trip_destinations = df2['Trip Destination'].str.split(',', expand=True).astype(float).rename(columns={0: 'lat', 1: 'lng'})

# Combine all locations and remove duplicates
all_locations = pd.concat([driver_locations, trip_origins, trip_destinations]).drop_duplicates()

print(f"Total unique locations: {len(all_locations)}")
print(all_locations.head())


Total unique locations: 406747
        lat       lng
0  6.602207  3.270465
1  6.592097  3.287445
2  6.596133  3.281784
3  6.596142  3.280526
4  6.609232  3.288800


In [9]:
# Extract unique locations
unique_locations = df1[['lat', 'lng']].drop_duplicates().reset_index(drop=True)

# Initialize the Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

# Function to reverse geocode
def reverse_geocode(lat, lng):
    try:
        location = geolocator.reverse((lat, lng), exactly_one=True)
        return location.address
    except Exception as e:
        print(f"Error: {e}")
        return None

# Load cache if exists
cache_filename = 'geocode_cache.pkl'
try:
    with open(cache_filename, 'rb') as f:
        geocode_cache = pickle.load(f)
except FileNotFoundError:
    geocode_cache = {}

# Function to get region from address
def extract_region(address):
    if address:
        parts = address.split(", ")
        return parts[-4] if len(parts) > 3 else None
    return None

# Process in batches with caching
batch_size = 1000  # Define your batch size
for start in tqdm(range(0, len(unique_locations), batch_size)):
    end = min(start + batch_size, len(unique_locations))
    batch = unique_locations.iloc[start:end]
    
    for idx, row in batch.iterrows():
        lat, lng = row['lat'], row['lng']
        coord = (lat, lng)
        
        if coord in geocode_cache:
            address = geocode_cache[coord]
        else:
            address = reverse_geocode(lat, lng)
            geocode_cache[coord] = address
            time.sleep(1)  # Delay to avoid hitting rate limits
        
        unique_locations.at[idx, 'address'] = address
        unique_locations.at[idx, 'region'] = extract_region(address)
    
    # Save cache to file every batch
    with open(cache_filename, 'wb') as f:
        pickle.dump(geocode_cache, f)

# Save the results
unique_locations.to_csv('unique_locations_with_regions.csv', index=False)

print(unique_locations.head())


  0%|          | 0/211 [00:00<?, ?it/s]

Error: HTTP Error 403: Forbidden
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: HTTP Error 403: Forbidden
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: HTTP Error 403: Forbidden
Error: Service timed out
Error: HTTP Error 403: Forbidden
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service timed out
Error: Service tim

  0%|          | 0/211 [06:14<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Ensure Trip Start Time and Trip End Time are in datetime format
df2['Trip Start Time'] = pd.to_datetime(df2['Trip Start Time'])
df2['Trip End Time'] = pd.to_datetime(df2['Trip End Time'])

# Split Trip Origin and Trip Destination into latitude and longitude
df2[['Origin Lat', 'Origin Lng']] = df2['Trip Origin'].str.split(',', expand=True).astype(float)
df2[['Destination Lat', 'Destination Lng']] = df2['Trip Destination'].str.split(',', expand=True).astype(float)

# Extract additional time-based features
df2['Start Hour'] = df2['Trip Start Time'].dt.hour
df2['Start Day of Week'] = df2['Trip Start Time'].dt.dayofweek

# Calculate trip duration in minutes
df2['Trip Duration'] = (df2['Trip End Time'] - df2['Trip Start Time']).dt.total_seconds() / 60

# Rename columns to match for merging
df1.rename(columns={'order_id': 'Trip ID'}, inplace=True)

# Merge the datasets on 'Trip ID'
df_merged = pd.merge(df1, df2, on='Trip ID')

df_merged

In [ ]:
# Ensure Trip Start Time and Trip End Time are in datetime format
df2['Trip Start Time'] = pd.to_datetime(df2['Trip Start Time'])
df2['Trip End Time'] = pd.to_datetime(df2['Trip End Time'])

# Split Trip Start Time into Trip Start Date and Trip Start Time
df2['Trip Start Date'] = df2['Trip Start Time'].dt.date
df2['Trip Start Time Only'] = df2['Trip Start Time'].dt.time

# Split Trip End Time into Trip End Date and Trip End Time
df2['Trip End Date'] = df2['Trip End Time'].dt.date
df2['Trip End Time Only'] = df2['Trip End Time'].dt.time

# Drop the original Trip Start Time and Trip End Time columns if not needed
df2.drop(columns=['Trip Start Time', 'Trip End Time'], inplace=True)

# Display the updated DataFrame
df2.head()

In [ ]:
# Extract time of day feature
def get_time_of_day(hour):
    if hour < 6:
        return 'Night'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'

df_merged['Time of Day'] = df_merged['Start Hour'].apply(get_time_of_day)

# Convert 'Time of Day' to categorical feature
df_merged['Time of Day'] = df_merged['Time of Day'].astype('category')